In [ ]:
# IN THIS FILE, PLANNING AND CREATION OF INTERACTIONS DATA BETWEEN EACH PAIR OF COWS IS DONE FOR EACH TIMESTAMP
# INTERACTIONS.CSV FILE IS CREATED IN THIS NOTEBOOK
#     CONTAINS THE INTERACTING PAIR OF COWS FOR EACH TIMESTAMP CONSIDERED

In [ ]:
# STEPS PLANNED TO DO
# 1. ITERATE OVER EACH TIMESTAMP OF COWS AND FOR EACH PAIR OF COWS, FIND THE EUCLIDEAN DISTANCE
#     DECIDE A THRESHOLD FOR EUCLIDEAN DISTANCE, LESS THAN WHICH A PROXIMITY IS CONSIDERED BETWEEN TWO COWS FOR INTERACTION
#     ASSIGN THE DATA BETWEEN EACH PAIR AS 1 IF THEY ARE IN CLOSE PROXIMITY, ELSE 0
# 2. DEFINE INTERACTION BETWEEN EACH PAIR OF COWS AS, 
#     IF THE CLOSE PROXIMITY BETWEEN THE PAIR IS 0, THEN INTERACTION IS 0 BY DEFAULT
#     IF THE CLOSE PROXIMITY BETWEEN THE PAIR IS 1, 
#         ASSIGN 1 FOR INTERACTION IF INTERACTION IN PREVIOUS TIMESTAMP IS 1, OR IF CLOSE PROXIMITY EXIST FOR 4 CONTINUOUS TIMESTAMPS CONSIDERING BOTH FORWARD AND BACKWARD TIMESTAMPS
#         ELSE ASSIGN 0 FOR INTERACTION

In [1]:
# IMPORT STATEMENTS
import pandas as pd
from collections import defaultdict
import os
from datetime import datetime
import pytz
import json
import numpy as np

In [ ]:
# 1. ITERATE OVER EACH TIMESTAMP OF COWS AND FOR EACH PAIR OF COWS, FIND THE EUCLIDEAN DISTANCE
#     DECIDE A THRESHOLD FOR EUCLIDEAN DISTANCE, LESS THAN WHICH A PROXIMITY IS CONSIDERED BETWEEN TWO COWS FOR INTERACTION
#     ASSIGN THE DATA BETWEEN EACH PAIR AS 1 IF THEY ARE IN CLOSE PROXIMITY, ELSE 0

In [2]:
# READ THE TIMESTAMP COW COORDINATES DICT FROM JSON FILE
with open("TimeStampCowCoordinates.json", "r") as file:
    timestampCowCoordinates = json.load(file)

In [3]:
# GET THE SORTED LIST OF TIMESTAMPS
timestamps = list(timestampCowCoordinates.keys())
timestamps.sort()

In [10]:
# FUNCTION TO FIND THE X, Y EUCLIDEAN DISTANCE
def findEuclideanDistance(pos1, pos2):
    if pos1 is None or pos2 is None:
        return None
    
    x1, y1 = pos1[:2]
    x2, y2 = pos2[:2]
    
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [35]:
# PRESET THE THRESHOLD FOR INTERACTION
# THE THRESHOLD IS SET TO 2.5 METERS, OR 250 (CMS) AS UTILIZED IN THE STUDY PROXIMITY BASED INTERACTIONS
THRESHOLD_DISTANCE = 250

In [36]:
# ITERATING OVER EACH PAIR OF COWS DURING EACH TIMESTAMP AND DETERMINING INTERACTIONS BASED ON THRESHOLD DISTANCE
timestampProximity = defaultdict(set)
for timestamp, positions in timestampCowCoordinates.items():
    proximitySet = timestampProximity[timestamp]
    for i in range(len(positions)-1):
        for j in range(i+1, len(positions)):
            iPosition = positions[i]
            jPosition = positions[j]
            distance = findEuclideanDistance(iPosition, jPosition)
            if distance is not None and distance <= THRESHOLD_DISTANCE:
                proximitySet.add((i, j))

In [ ]:
# 2. DEFINE INTERACTION BETWEEN EACH PAIR OF COWS AS, 
#     IF THE CLOSE PROXIMITY BETWEEN THE PAIR IS 0, THEN INTERACTION IS 0 BY DEFAULT
#     IF THE CLOSE PROXIMITY BETWEEN THE PAIR IS 1, 
#         ASSIGN 1 FOR INTERACTION IF INTERACTION IN PREVIOUS TIMESTAMP IS 1, OR IF CLOSE PROXIMITY EXIST FOR 4 CONTINUOUS TIMESTAMPS CONSIDERING BOTH FORWARD AND BACKWARD TIMESTAMPS
#         ELSE ASSIGN 0 FOR INTERACTION

In [37]:
# FUNCTION TO CONFIRM IF PROXIMITY EXISTS IN FOUR CONTINUOUS TIMESTAMPS
def confirmProximity(timestampIndex, timestamps, timestampProximity, timestampInteraction, THRESHOLD_DISTANCE, i, j):
    curr = 1
    currTimestampIndex = timestampIndex - 1
    if currTimestampIndex >= 0:
        timestamp = timestamps[currTimestampIndex]
        if (i, j) in timestampInteraction[timestamp]:
            return True
    while currTimestampIndex > 0: 
        timestamp = timestamps[currTimestampIndex]
        if (i, j) in timestampProximity[timestamp]:
            curr += 1
            if curr >= 4:
                break
            currTimestampIndex -= 1
        else:
            break
    if curr >= 4:
        return True
    currTimestampIndex = timestampIndex + 1
    while currTimestampIndex < len(timestamps): 
        timestamp = timestamps[currTimestampIndex]
        if (i, j) in timestampProximity[timestamp]:
            curr += 1
            if curr >= 4:
                break
            currTimestampIndex += 1
        else:
            break
    return curr >= 4

In [38]:
# INITIALIZING THE TIMESTAMP INTERACTION SET
timestampInteraction = defaultdict(set)
for ind, timestamp in enumerate(timestamps):
    proximitySet = timestampProximity[timestamp]
    interactionSet = timestampInteraction[timestamp]
    for i, j in proximitySet:
        if confirmProximity(ind, timestamps, timestampProximity, timestampInteraction, THRESHOLD_DISTANCE, i, j):
            interactionSet.add((i, j))

In [39]:
# FIND THE TOTAL NO. OF INTERACTION BETWEEN EACH PAIR OF COWS IN THE ENTIRE DURATION
interactions = defaultdict(int)
for timestamp in timestamps:
    interactionSet = timestampInteraction[timestamp]
    for i, j in interactionSet:
        interactions[(i, j)] += 1

In [44]:
# CREATE THE LIST OF EDGES AND STORE THEM AS CSV FILE TO ACCES USING R 
edges = []
for (i, j), interactionCount in interactions.items():
    edges.append([i+1, j+1, interactionCount])
df = pd.DataFrame(edges, columns=["Vertex1", "Vertex2", "Weight"])
csv_file = "Interactions.csv"
df.to_csv(csv_file, index=False)